In [1]:
import os
import pandas as pd

test_path = "/Users/giovanna/Desktop/NTNU/TextAnalytics/Project/test/try1.csv"
columns = ["comment"]
df = pd.read_csv(test_path, header=None, names=columns, encoding="latin1")

In [2]:
#df.drop('id', axis=1, inplace=True)
df.head()

,comment
0,comment;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
1,pog;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
2,@Nightblue3 HI;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;...
3,AYAYA;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
4,Na is idiot;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;...


In [3]:
from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk.tokenize import TweetTokenizer
import spacy
import re

STOP = stopwords.words("english")

twitter_words_to_filter = ["rt"]
def filter_single_letter(word):
    return len(word) > 1 or word == 'a' or word == 'i'

tt = TweetTokenizer()
def twitter_preprocessing(post):
    tokenized_post = tt.tokenize(post)
    pattern = re.compile(r"^[a-zA-Z\!\?\,\.\']+$")
    dot_pattern = re.compile(r"\.{2,}")
    url_matcher = re.compile(r"^[a-zA-Z]+\.[a-zA-Z]+$")
    tokens = [
        w.lower() for w in tokenized_post
        if pattern.match(w) and not dot_pattern.match(w)
        and w.lower() not in twitter_words_to_filter
        and filter_single_letter(w.lower())
        and not url_matcher.match(w.lower())
    ]
    return " ".join(tokens).strip()



In [4]:
import psutil
total_gb_ram = int(psutil.virtual_memory().total / 1024 / 1024 / 1024)
print(f"{total_gb_ram} GB RAM")

16 GB RAM


In [6]:
if total_gb_ram >= 16:
    import mapply
    mapply.init()

    df["comment"] = df["comment"].mapply(twitter_preprocessing, df["comment"].tolist())
else:
    df["comment"] = df["comment"].apply(twitter_preprocessing)

  0%|          | 0/80 [00:00<?, ?it/s]

In [7]:
df.head()

,comment
0,comment
1,pog
2,hi
3,ayaya
4,na is idiot


In [12]:
_path = "/Users/giovanna/Desktop/NTNU/TextAnalytics/Project"

In [13]:
save_path = os.path.join(_path, "try.csv")
df.to_csv(save_path, index=False)

In [10]:
df

,comment
0,comment
1,pog
2,hi
3,ayaya
4,na is idiot
...,...
48669,bye chat biblethump
48670,
48671,
48672,time please


In [11]:
label_columns = df.iloc[:,1:].columns
df['labels'] = df[label_columns].apply(lambda x: list(x), axis=1)
df.drop(label_columns, inplace=True, axis=1)

train_size = 0.8

train_df = df.sample(frac=train_size, random_state=42)
val_df = df.drop(train_df.index).reset_index(drop=True)
train_df.reset_index(inplace=True, drop=True)

/var/folders/hx/s2fpl8513bj8s4tk49n3h1lc0000gn/T/ipykernel_68375/2786037225.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['labels'] = df[label_columns].apply(lambda x: list(x), axis=1)


ValueError: Length of values (0) does not match length of index (48674)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class MultiLabelDataset(Dataset):
    
    def __init__(self, df, tokenizer, max_len, new_data=False):
        self.tokenizer = tokenizer
        self.data = df
        self.text = df.comment_text
        self.new_data = new_data
        self.max_len = max_len
        
        if not new_data:
            self.targets = self.data.labels
            
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, index):
        text = str(self.text[index])
        
        inputs = self.tokenizer.encode_plus(text,None,
                                           add_special_tokens=True,
                                           max_length=self.max_len,
                                           pad_to_max_length=True,
                                           return_token_type_ids=True)
        out = {
            "input_ids": torch.tensor(inputs['input_ids'], dtype=torch.long),
            "attention_mask": torch.tensor(inputs['attention_mask'], dtype=torch.long),
            "token_type_ids": torch.tensor(inputs['token_type_ids'], dtype=torch.long)
        }
        if not self.new_data:
            out["targets"] = torch.tensor(self.targets[index], dtype=torch.float)
            
        return out